In [ ]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import matplotlib.pyplot as plt
import pandas as pd

Step 1: Visit the Website
Use automated browsing to visit the Mars Temperature Data Site. Inspect the page to identify which elements to scrape. Note that the URL is https://data-class-mars-challenge.s3.amazonaws.com/Mars/index.html.

Hint To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.

In [ ]:
# Visit the website
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
url = "https://data-class-mars-challenge.s3.amazonaws.com/Mars/index.html"

Step 2: Scrape the Table
Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas read_html function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [ ]:
# Create a Beautiful Soup Object
soup = BeautifulSoup(browser.html, 'html.parser')

In [ ]:
# Extract all rows of data
results = soup.find_all('table', class_= 'table')

Step 3: Store the Data
Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

id: the identification number of a single transmission from the Curiosity rover
terrestrial_date: the date on Earth
sol: the number of elapsed sols (Martian days) since Curiosity landed on Mars
ls: the solar longitude
month: the Martian month
min_temp: the minimum temperature, in Celsius, of a single Martian day (sol)
pressure: The atmospheric pressure at Curiosity's location

In [ ]:
# Create an empty list
collection = []

# Loop through the scraped data to create a list of rows
for i in results:
    j = i.find_all('tr')
    row = [item.text for item in j]
    collection.append(row)  

In [ ]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
# Create a Pandas DataFrame by using the list of rows and a list of the column names
tables = pd.read_html(url)
mars_df = tables[0]

In [ ]:
# Confirm DataFrame was created successfully
mars_df

id	terrestrial_date	sol	ls	month	min_temp	pressure
0	2	2012-08-16	10	155	6	-75.0	739.0
1	13	2012-08-17	11	156	6	-76.0	740.0
2	24	2012-08-18	12	156	6	-76.0	741.0
3	35	2012-08-19	13	157	6	-74.0	732.0
4	46	2012-08-20	14	157	6	-74.0	740.0
...	...	...	...	...	...	...	...
1862	1889	2018-02-23	1973	133	5	-78.0	730.0
1863	1892	2018-02-24	1974	134	5	-77.0	729.0
1864	1894	2018-02-25	1975	134	5	-76.0	729.0
1865	1893	2018-02-26	1976	135	5	-77.0	728.0
1866	1895	2018-02-27	1977	135	5	-77.0	727.0
1867 rows × 7 columns

Step 4: Prepare Data for Analysis
Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate datetime, int, or float data types.

Hint You can use the Pandas astype and to_datetime methods to accomplish this task.

In [ ]:
# Examine data type of each column
mars_df.dtypes

id                    int64
terrestrial_date     object
sol                   int64
ls                    int64
month                 int64
min_temp            float64
pressure            float64
dtype: object

In [ ]:
# Change data types for data analysis
mars_df['terrestrial_date'] = pd.to_datetime(mars_df['terrestrial_date'])
mars_df['id'] = mars_df['id'].astype(object)

In [ ]:
# Confirm type changes were successful by examining data types again
mars_df.dtypes

id                          object
terrestrial_date    datetime64[ns]
sol                          int64
ls                           int64
month                        int64
min_temp                   float64
pressure                   float64
dtype: object

Step 5: Analyze the Data
Analyze your dataset by using Pandas functions to answer the following questions:

How many months exist on Mars?
How many Martian (and not Earth) days worth of data exist in the scraped dataset?
What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
Find the average the minimum daily temperature for all of the months.
Plot the results as a bar chart.
Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
Find the average the daily atmospheric pressure of all the months.
Plot the results as a bar chart.
About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
Consider how many days elapse on Earth in the time that Mars circles the Sun once.
Visually estimate the result by plotting the daily minimum temperature.

In [ ]:
#1. How many months are there on Mars?
mars_df.groupby(['month']).nunique()["terrestrial_date"].count

<bound method Series.count of month
1     174
2     178
3     192
4     194
5     149
6     147
7     142
8     141
9     134
10    112
11    138
12    166
Name: terrestrial_date, dtype: int64>

In [ ]:
#2. How many Martian days' worth of data are there?
mars_df['terrestrial_date'].count()

1867

In [ ]:
# 3. What is the average low temperature by month?
avg_temp =mars_df.groupby(['month']).mean()['min_temp']
avg_temp

month
1    -77.160920
2    -79.932584
3    -83.307292
4    -82.747423
5    -79.308725
6    -75.299320
7    -72.281690
8    -68.382979
9    -69.171642
10   -71.982143
11   -71.985507
12   -74.451807
Name: min_temp, dtype: float64

In [ ]:
# Plot the average temperature by month
avg_temp.plot.bar()
plt.xlabel('month')
plt.ylabel('temperature (C)')

In [ ]:
# Identify the coldest and hottest months in Curiosity's location
order_temp = avg_temp.sort_values()
order_temp.plot.bar()
plt.xlabel('month')
plt.ylabel('temperature (C)')

In [ ]:
# 4. Average pressure by Martian month
avg_pressure =mars_df.groupby(['month']).mean()['pressure']
avg_pressure

month
1     862.488506
2     889.455056
3     877.322917
4     806.329897
5     748.557047
6     745.054422
7     795.105634
8     873.829787
9     913.305970
10    887.312500
11    857.014493
12    842.156627

In [ ]:
# Plot the average pressure by month
ascend_avg_pressure = avg_pressure.sort_values()
ascend_avg_pressure.plot.bar()
plt.xlabel('month')
plt.ylabel('pressure')

In [ ]:
# 5. How many terrestrial (earth) days are there in a Martian year?
martian_days = "687"
martian_days

'687'

In [ ]:
mars_df['min_temp'].plot(
    kind='line', 
    xlabel='Number of Terrestrial Days', 
    ylabel='Minimum Temperature')
plt.show()

Findings and conclusions:

The third month has the coldest temperature on th eplanet Mars, while the eighth month is considered the warmest. However it is considered to be very cold when looking at the planet Earth. 

The atmospheric pressure on average seemes to be the lowest in the sixth month and at its highest in the ninth.

The distance from peak to peak is roughly 1425-750, or 675 days, a year on Mars appears to be about 675 days. When doing a quick search on the Internet, we can conclude that a Mars year is equivalent to 687 Earth days.

Step 6: Save the Data
Export the DataFrame to a CSV file.

In [ ]:
# Write the data to a CSV
mars_df.to_csv("mars_weather_data.csv")

In [ ]:
browser.quit()